In [1]:
import sys
sys.path.append('../src')
from evaluation import rename_string, eval_results

In [2]:
import wandb
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from evaluate import get_results
from omegaconf import OmegaConf
from main import load_data, make_model
import numpy as np
import os
import glob
import demap
from transformations import NonTransform

specific to @xingzhis' model: load data names from config. 

In [3]:
# Initialize wandb (replace 'your_entity' and 'your_project' with your specific details)
wandb.login()
api = wandb.Api()

# Specify your entity, project, and sweep ID
entity = "xingzhis"
project = "dmae"
sweep_id = 'wgysuau8'

# Fetch the sweep
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

run_ids = [run.id for run in sweep.runs]
run = sweep.runs[0] # for example.
cfg = OmegaConf.create(run.config)
folder_path = "../src/wandb/"
try:
    folder_list = glob.glob(f"{folder_path}*{run.id}*")
    ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
    ckpt_path = ckpt_files[0]
except:
    print(f"No checkpoint found for run {run.id}")
cfg = OmegaConf.create(run.config)

data_path = os.path.join(cfg.data.root, cfg.data.name + cfg.data.filetype)
noiseless_path = os.path.join(cfg.data.root, rename_string(cfg.data.name) + cfg.data.filetype)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


Specific to @xingzhis' model: load model

In [6]:
pp = NonTransform()
emb_dim = cfg.model.emb_dim
dist_std = 1.
input_dim = 100
model = make_model(cfg, input_dim, emb_dim, pp, dist_std, from_checkpoint=True, checkpoint_path=ckpt_path)

compute metric

In [7]:
result = eval_results(data_path, noiseless_path, model)

In [8]:
result

{'demap': 0.7140251186997651,
 'accuracy': 0.7504084617128052,
 'gene_corr_mse': nan,
 'seedmethod': 'groups,46',
 'bcv': '0.2',
 'dropout': '0.7'}